## Part 1 - Extraction and Frequencies

In [11]:
import nltk
from nltk.collocations import *
from datetime import datetime

In [12]:
extract_start = datetime.now()

file = open("Corpus/maltese-1.txt")
corpus = file.read()

tokens = nltk.word_tokenize(corpus)
extract_end = datetime.now()

extraction_time = dict()
extraction_time['extraction_time'] = extraction_end - extraction_start
print('extraction_time(HH::MM:SS:ms) - {}\n\n'.format(extraction_time['extraction_time']))
print(tokens)

extraction_time(HH::MM:SS:ms) - 0:00:00.460885


['<', 'text', 'id=', "''", 'academic1', "''", '>', '<', 'p', 'id=', "''", '0', "''", '>', '<', 's', 'id=', "''", '0', "''", '>', 'L-', 'DEF', 'il-', 'null', 'gÄ§an', 'NOUN', 'gÄ§an', 'null', 'prinÄ‹ipali', 'ADJ', 'prinÄ‹ipali', 'null', "ta'", 'GEN', "ta'", 'null', 'Conectando', 'NOUN-PROP', 'Conectando', 'null', 'Mundos', 'NOUN-PROP', 'Mundos', 'null', '(', 'X-PUN', '(', 'null', 'Malta', 'NOUN-PROP', 'Malta', 'null', ')', 'X-PUN', ')', 'null', '(', 'X-PUN', '(', 'null', 'CMM', 'X-ABV', 'CMM', 'null', ')', 'X-PUN', ')', 'null', ',', 'X-PUN', ',', 'null', 'programm', 'NOUN', 'programm', 'null', "ta'", 'GEN', "ta'", 'null', 'Edukazzjoni', 'NOUN', 'Edukazzjoni', 'null', "b'", 'PREP', 'bi', 'null', 'RiÅ¼q', 'NOUN', 'risq', 'null', 'ÄŠittadinanza', 'NOUN', 'ÄŠittadinanza', 'null', 'Globali', 'ADJ', 'Globali', 'null', '(', 'X-PUN', '(', 'null', 'Global', 'NOUN-PROP', 'Global', 'null', 'Citizenship', 'NOUN-PROP', 'Citizenship', 'null', 'Educatio

In [13]:
bigrams = nltk.bigrams(tokens)

fdist = nltk.FreqDist(bgs)
for k,v in fdist.items():
    print (k,v)

('<', 'text') 11
('text', 'id=') 11
('id=', "''") 1404
("''", 'academic1') 1
('academic1', "''") 1
("''", '>') 1404
('>', '<') 1730
('<', 'p') 316
('p', 'id=') 316
("''", '0') 22
('0', "''") 22
('<', 's') 1077
('s', 'id=') 1077
('>', 'L-') 117
('L-', 'DEF') 122
('DEF', 'il-') 3077
('il-', 'null') 3086
('null', 'gÄ§an') 35
('gÄ§an', 'NOUN') 35
('NOUN', 'gÄ§an') 35
('gÄ§an', 'null') 35
('null', 'prinÄ‹ipali') 9
('prinÄ‹ipali', 'ADJ') 10
('ADJ', 'prinÄ‹ipali') 10
('prinÄ‹ipali', 'null') 10
('null', "ta'") 703
("ta'", 'GEN') 836
('GEN', "ta'") 843
("ta'", 'null') 1935
('null', 'Conectando') 5
('Conectando', 'NOUN-PROP') 3
('NOUN-PROP', 'Conectando') 3
('Conectando', 'null') 5
('null', 'Mundos') 5
('Mundos', 'NOUN-PROP') 4
('NOUN-PROP', 'Mundos') 4
('Mundos', 'null') 5
('null', '(') 127
('(', 'X-PUN') 149
('X-PUN', '(') 149
('(', 'null') 149
('null', 'Malta') 13
('Malta', 'NOUN-PROP') 13
('NOUN-PROP', 'Malta') 13
('Malta', 'null') 13
('null', ')') 143
(')', 'X-PUN') 149
('X-PUN', ')') 149
(

("''", 'X-PUN') 262
('X-PUN', "''") 262
("''", 'null') 262
('null', 'kien') 47
('kien', 'KIEN') 53
('null', 'wieÄ§ed') 24
('wieÄ§ed', 'NUM-WHD') 24
('NUM-WHD', 'wieÄ§ed') 51
('wieÄ§ed', 'null') 51
('null', 'tajjeb') 7
('tajjeb', 'ADV') 3
('ADV', 'tajjeb') 3
('tajjeb', 't-j-b') 23
('t-j-b', 'Ä§afna') 3
('Ä§afna', 'ADV') 28
('ADV', 'Ä§af') 28
('Ä§af', 'Ä§-w-f') 28
('Ä§-w-f', 'biex') 1
('null', 'nagÄ§mlu') 5
('nagÄ§mlu', 'VERB') 6
('gÄ§-m-l', 'xebh') 1
('xebh', 'NOUN') 2
('NOUN', 'xebh') 2
('xebh', 'x-b-h') 2
('x-b-h', 'bejn') 1
('bejn', 'PREP') 54
('PREP', 'bejn') 54
('b-j-n', 'ix-') 2
('ix-', 'DEF') 22
('null', 'xena') 3
('xena', 'NOUN') 3
('NOUN', 'xena') 3
('xena', 'x-n-j') 3
('x-n-j', 'Maltija') 1
('Maltija', 'ADJ') 2
('null', 'dik') 22
('dik', 'PRON-DEM') 25
('null', 'fin-') 10
('fin-', 'PREP-DEF') 13
('null', 'Nofsinhar') 36
('Nofsinhar', 'NOUN') 35
('NOUN', 'Nofsinhar') 35
('Nofsinhar', 'null') 36
('null', 'tad-') 39
('tad-', 'GEN-DEF') 48
('null', 'dinja') 71
('dinja', 'NOUN') 72

('QUAN', 'bosta') 13
('bosta', 'null') 13
('null', 'nazzjonijiet') 1
('nazzjonijiet', 'NOUN') 1
('NOUN', 'nazzjon') 1
('nazzjon', 'null') 1
('>', 'Ir-') 6
('Ir-', 'DEF') 6
('null', 'rapport') 3
('rapport', 'NOUN') 3
('NOUN', 'rapport') 3
('rapport', 'null') 3
("Ä¡-j-'", 'aÄ¡Ä¡ornat') 1
('aÄ¡Ä¡ornat', 'VERB') 1
('VERB', 'aÄ¡Ä¡ornat') 1
('aÄ¡Ä¡ornat', 'null') 2
('null', 'jinkludi') 1
('jinkludi', 'VERB') 1
('null', 'kwantitattiva') 1
('kwantitattiva', 'ADJ') 1
('ADJ', 'kwantitattiv') 1
('kwantitattiv', 'null') 1
('null', 'kwalitattiva') 1
('kwalitattiva', 'ADJ') 1
('ADJ', 'kwalitattiv') 1
('kwalitattiv', 'null') 1
('null', 'jinÄ¡abru') 3
('jinÄ¡abru', 'VERB') 4
('VERB', 'nÄ¡abar') 4
('nÄ¡abar', 'Ä¡-b-r') 4
('Ä¡-b-r', 'fl-') 1
('null', 'IndiÄ‹i') 1
('IndiÄ‹i', 'ADJ') 1
('ADJ', 'IndiÄ‹i') 1
('IndiÄ‹i', 'null') 1
('null', 'IÅ') 1
('IÅ', '»') 2
('»', 'B') 2
('B', 'X-ABV') 1
('X-ABV', 'IÅ') 1
('B', 'null') 1
('null', 'kuntrarju') 4
('kuntrarju', 'NOUN') 4
('NOUN', 'kuntrarju') 4
('kuntrarju',

('null', 'rimi') 2
('rimi', 'NOUN') 2
('NOUN', 'rimi') 2
('rimi', 'r-m-j') 2
('r-m-j', 'bi') 1
('bi', 'PREP') 14
('Ä§-s-r', 'lis-') 1
('lis-', 'LIL-DEF') 3
('null', 'ferm') 6
('null', 'ikbar') 13
('ikbar', 'ADJ') 17
('k-b-r', 'minn') 3
('null', 'PQJ') 13
('PQJ', 'X-ABV') 13
('X-ABV', 'PQJ') 13
('PQJ', 'null') 13
("''", '29') 13
('29', "''") 13
('null', 'isforz') 1
('isforz', 'NOUN') 1
('NOUN', 'isforz') 2
('isforz', 'null') 2
('null', 'jitraÅ¼Å¼nu') 3
('jitraÅ¼Å¼nu', 'VERB') 3
('VERB', 'traÅ¼Å¼an') 4
('traÅ¼Å¼an', 'r-Å¼-n') 4
('r-Å¼-n', 'l-') 4
('null', 'dannu') 4
('dannu', 'NOUN') 4
('NOUN', 'dann') 4
('dann', 'd-n-n') 10
('d-n-n', 'gÄ§all-') 4
('null', 'irid') 7
('irid', 'VERB') 7
('r-j-d', 'ikun') 1
('null', 'proporzjonali') 1
('proporzjonali', 'ADJ') 1
('ADJ', 'proporzjonali') 1
('proporzjonali', 'null') 1
('null', 'Ä§sarat') 4
('Ä§sarat', 'NOUN') 4
('NOUN', 'Ä§sara') 4
('Ä§sara', 'null') 4
('null', 'jikkawÅ¼a') 3
('jikkawÅ¼a', 'VERB') 3
('VERB', 'kkawÅ¼a') 3
('kkawÅ¼a', 'null') 3


('null', 'radikali') 1
('radikali', 'NOUN') 1
('NOUN', 'radikali') 1
('radikali', 'null') 1
('r-Å¼-n', 'tassoÄ‹jah') 1
('tassoÄ‹jah', 'VERB') 1
('null', 'mat-') 2
('mat-', 'PREP-DEF') 4
('n-q-s', 'fil-') 1
('null', 'kummerÄ‹jalizzazzjoni') 3
('kummerÄ‹jalizzazzjoni', 'NOUN') 3
('NOUN', 'kummerÄ‹jalizzazzjoni') 3
('kummerÄ‹jalizzazzjoni', 'null') 3
("''", '61') 8
('61', "''") 8
('null', 'igawdih') 1
('igawdih', 'VERB') 1
('VERB', 'gawda') 7
('gawda', 'null') 4
("Ä¡-j-'", 'pprivatizzat') 1
('pprivatizzat', 'PART-PASS') 1
('PART-PASS', 'ipprivatizzat') 2
('ipprivatizzat', 'null') 2
('null', 'dritt') 4
('dritt', 'ADV') 2
('ADV', 'dritt') 2
("''", '62') 8
('62', "''") 8
('r-Å¼-n', 'tat-') 1
('s-t-gÄ§', 'jgÄ§in') 1
('jgÄ§in', 'VERB') 11
('gÄ§-j-n', 'fit-') 1
('null', 'forom') 12
('forom', 'NOUN') 17
('null', 'formazzjoni') 1
('formazzjoni', 'NOUN') 1
('NOUN', 'formazzjoni') 1
('formazzjoni', 'null') 1
('null', 'xbiek') 1
('xbiek', 'NOUN') 1
('NOUN', 'xibka') 9
('xibka', 'null') 9
('null', 'e

...

('NOUN-PROP', 'Italia') 2
('Italia', 'null') 2
('inizzjattiva', 'ADJ') 1
('ADJ', 'inizzjattiva') 1
('Taljana', 'NOUN') 1
('NOUN', 'taljan') 1
('FLO', 'X-ABV') 2
('X-ABV', 'FLO') 2
("''", '173') 1
('173', "''") 1
('null', 'biÄ‹-') 1
('biÄ‹-', 'PREP-DEF') 2
('null', 'jinkitbu') 1
('jinkitbu', 'VERB') 1
('k-t-b', "f'") 1
('null', 'apposta') 1
('apposta', 'ADV') 1
('ADV', 'apposta') 1
('apposta', 'null') 1
('null', 'jinqasmu') 1
('jinqasmu', 'VERB') 1
('q-s-m', 'skond') 1
("''", '174') 1
('174', "''") 1
('x-w-q', 'jbigÄ§u') 1
('b-j-gÄ§', 'prodotti') 1
('null', 'liÄ‹enzja') 3
('liÄ‹enzja', 'NOUN') 3
("''", '175') 1
('175', "''") 1
('>', 'â€') 1
('s-t-gÄ§', 'gÄ§aldaqstant') 1
("''", '176') 1
('176', "''") 1
('null', 'jÄ§allsu') 1
('x-r-j', 'prodott') 1
('b-j-gÄ§', 'awtorizzati') 1
('awtorizzati', 'PART-PASS') 1
('PART-PASS', 'awtorizzat') 1
('awtorizzat', 'null') 1
('null', 'VLO') 1
('VLO', 'X-ABV') 1
('X-ABV', 'VLO') 1
('VLO', 'null') 1
("''", '177') 1
('177', "''") 1
('s-t-gÄ§', 'huwa') 1


('NOUN', 'BiodiversitÃ') 1
('BiodiversitÃ', 'null') 1
('s-t-gÄ§', 'jintuÅ¼aw') 1
('null', 'vittma') 1
('vittma', 'NOUN') 1
('null', 'tibdil') 1
('tibdil', 'NOUN') 1
('tibdil', 'd-b-l') 1
('d-b-l', 'fil-') 1
('r-b-t', 'id-') 1
('null', 'destin') 1
('destin', 'NOUN') 1
('NOUN', 'destin') 1
('destin', 'null') 1
('null', 'tbati') 1
('tbati', 'NOUN') 1
('NOUN', 'bata') 1
('b-t-j', 'mill-') 1
('null', 'esportati') 1
('esportati', 'PART-PASS') 1
('PART-PASS', 'esportat') 1
('esportat', 'null') 1
('r-j-d', 'in-') 1
('b-q-gÄ§', 'tad-') 1
('gÄ§-m-l', 'diversi') 1
('null', 'talbiet') 2
('talbiet', 'NOUN') 3
('null', 'lis-') 1
('ifqar', 'VERB') 1
('VERB', 'fqir') 1
('f-q-r', 'pajjiÅ¼i') 1
('r-j-d', 'jiddefendu') 1
('null', 'jbaxxux') 1
('jbaxxux', 'VERB') 1
('b-x-j', 'id-') 1
('null', 'Å¼ejjed') 1
('Å¼ejjed', 'ADJ') 1
('null', 'barranija') 2
('barranija', 'ADJ') 2
('b-r-r', '.') 2
('null', 'privileÄ¡Ä¡jati') 1
('privileÄ¡Ä¡jati', 'ADJ') 1
('ADJ', 'privileÄ¡Ä¡jat') 1
('privileÄ¡Ä¡jat', 'null') 1
('

('NOUN', 'telf') 1
('telf', 't-l-f') 1
('t-l-f', "ta'") 1
('f-w-q', 'setturi') 1
('setturi', 'NOUN') 3
('>', 'Sors') 1
('Sors', 'NOUN') 1
('NOUN', 'Sors') 1
('Sors', 'null') 1
('q-w-j', 'lejn') 1
("'-Ä§-d", 'd-') 1
('informali', 'NOUN') 1
('NOUN', 'informali') 1
('s-j-r', 'n-') 1
('f-h-m', '(') 1
('null', 'funzjonarji') 1
('funzjonarji', 'NOUN') 1
('NOUN', 'funzjonarju') 1
('funzjonarju', 'null') 1
('null', 'jirrappreÅ¼entawx') 1
('jirrappreÅ¼entawx', 'VERB') 1
('null', 'jiddeÄ‹iedi') 1
('jiddeÄ‹iedi', 'VERB') 1
('null', 'kurituri') 1
('kurituri', 'NOUN') 1
('NOUN', 'kuritur') 1
('kuritur', 'null') 1
('f-w-q', 'Ä§afna') 1
('null', 'bÄ§as-') 1
('bÄ§as-', 'PREP-DEF') 1
('null', 'mediÄ‹ini') 1
('mediÄ‹ini', 'NOUN') 1
('NOUN', 'mediÄ‹ina') 1
('mediÄ‹ina', 'null') 1
('null', 'kunsens') 2
('kunsens', 'NOUN') 2
('NOUN', 'kunsens') 2
('kunsens', 'null') 2
("Ä¡-j-'", 'miftiehma') 1
('miftiehma', 'PART-PASS') 1
('PART-PASS', 'miftiehem') 1
('miftiehem', 'f-h-m') 1
('f-h-m', 'Ä§afna') 1
('null', 

('b-d-j', 'x-') 1
('null', 'tiswija') 1
('tiswija', 'NOUN') 1
('NOUN', 'tiswija') 1
('tiswija', 'null') 1
('null', 'mormi') 1
('mormi', 'PART-PASS') 1
('PART-PASS', 'mormi') 1
('mormi', 'r-m-j') 1
('r-m-j', 'u') 1
('null', 'riÄ‹iklat') 1
('riÄ‹iklat', 'PART-PASS') 1
('PART-PASS', 'riÄ‹iklat') 1
('riÄ‹iklat', 'null') 1
('q-x-r', 'oÄ§ra') 1
('b-Ä§-r', ',') 2
('s-j-r', 'biss') 1
('>', 'Domna') 1
('Domna', 'NOUN-PROP') 1
('NOUN-PROP', 'Domna') 1
('Domna', 'null') 1
('r-Ä¡-gÄ§', 'jsalpa') 1
('jsalpa', 'VERB') 1
('s-l-p-j', ',') 1
('null', 'urietna') 2
('urietna', 'VERB') 2
("r-'-j", 'kemm') 3
('null', 'Ä§olma') 1
('Ä§olma', 'NOUN') 1
('NOUN', 'Ä§olm') 1
('Ä§olm', 'Ä§-l-m') 1
('Ä§-l-m', 'tagÄ§milha') 1
('gÄ§-m-l', 'realtÃ') 1
('null', 'ikollok') 1
('ikollok', 'VERB-PSEU') 1
('null', 'jitgÄ§allmu') 1
('jitgÄ§allmu', 'VERB') 1
('gÄ§-l-m', 'flimkien') 1
('null', 'sponsors') 1
('sponsors', 'NOUN') 1
('NOUN', 'sponsor') 1
('sponsor', 'null') 1
('null', 'wettaq') 1
('wettaq', 'VERB') 1
('null', 'F

('immaÄ¡inajt', 'VERB') 1
('b-Ä§-r', 'kien') 1
('Ä§-t-Ä¡', 'is-') 1
('null', 'temp') 1
('temp', 'NOUN') 1
('NOUN', 'temp') 1
('temp', 'null') 1
('null', 'kiefer') 1
('kiefer', 'PART-ACT') 1
('PART-ACT', 'kiefer') 1
('kiefer', 'k-f-r') 1
('k-f-r', ',') 1
('null', 'imdorrija') 1
('imdorrija', 'PART-PASS') 1
('PART-PASS', 'imdorri') 1
('imdorri', 'null') 1
('d-w-m', 'hemm') 1
('null', 'Ä§assejtni') 1
('Ä§assejtni', 'VERB') 1
('Ä§-s-s', 'tajba') 1
('t-j-b', 'nagÄ§mel') 1
('gÄ§-m-l', 'dan') 1
('>', 'Forsi') 1
('Forsi', 'ADV') 1
('null', 'nirringrazzja') 1
('nirringrazzja', 'VERB') 1
('VERB', 'rringrazzja') 1
('rringrazzja', 'null') 1
('null', 'Pasi') 1
('Pasi', 'NOUN') 1
('NOUN', 'Pasi') 1
('Pasi', 'null') 1
('null', 'Thomas') 1
('Thomas', 'NOUN-PROP') 1
('NOUN-PROP', 'Thomas') 1
('Thomas', 'null') 1
('null', 'Merienke') 1
('Merienke', 'NOUN-PROP') 1
('NOUN-PROP', 'Merienke') 1
('Merienke', 'null') 1
('null', 'Yenko') 1
('Yenko', 'NOUN-PROP') 1
('NOUN-PROP', 'Yenko') 1
('Yenko', 'null') 1
(

In [68]:
from nltk.corpus.reader.bnc import BNCCorpusReader
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

In [69]:
# Instantiate the reader and the files to be read
bnc_reader = BNCCorpusReader(root="BNC/Texts", fileids=r'[A-K]/\w*/\w*\.xml')
fileids = ['aca/A6U.xml']

In [70]:
#building n-grams
tokens = BNCCorpusReader.words(bnc_reader, fileids=fileids)
bigrams = nltk.bigrams(tokens)

fdist = nltk.FreqDist(bigrams)
for k,v in fdist.items():
    print (k,v)

('BEING', 'DRAWN') 1
('DRAWN', 'TO') 1
('TO', 'AN') 1
('AN', 'IMAGE') 1
('IMAGE', 'Guy') 1
('Guy', 'Brett') 1
('Brett', 'Why') 1
('Why', 'do') 1
('do', 'certain') 1
('certain', 'images') 2
('images', 'matter') 1
('matter', 'to') 1
('to', 'one') 2
('one', ',') 1
(',', 'and') 118
('and', 'why') 2
('why', 'is') 1
('is', 'the') 36
('the', 'desire') 3
('desire', 'to') 3
('to', 'answer') 2
('answer', 'this') 1
('this', 'question') 2
('question', 'as') 2
('as', 'involuntary') 1
('involuntary', 'as') 1
('as', 'the') 28
('the', 'response') 2
('response', 'itself') 1
('itself', '?') 1
('?', 'Why') 2
('Why', 'does') 1
('does', 'it') 2
('it', 'seem') 1
('seem', 'important') 1
('important', 'that') 2
('that', 'the') 41
('the', 'answer') 1
('answer', 'should') 1
('should', 'have') 3
('have', 'some') 1
('some', '‘') 1
('‘', 'objective') 1
('objective', '’') 1
('’', 'quality') 1
('quality', 'about') 1
('about', 'it') 2
('it', ',') 8
(',', 'an') 10
('an', 'insight') 1
('insight', 'into') 1
('into', 'hi

('becomes', 'the') 1
('the', 'gorgeous') 1
('gorgeous', 'aristocrat') 1
('aristocrat', '.') 1
('The', 'non-aggressive') 1
('non-aggressive', 'angel-like') 1
('angel-like', 'pose') 1
('pose', ',') 2
('course', 'extremely') 1
('extremely', 'seductive') 1
('seductive', ',') 1
('which', 'makes') 2
('makes', 'the') 2
('the', 'threat') 1
('threat', 'of') 1
('of', 'force') 1
('force', 'oblique') 1
('oblique', ',') 1
('only', 'implied') 1
('implied', ',') 1
('if', 'a') 1
('beautiful', 'face') 1
('face', 'was') 1
('was', 'being') 1
('being', 'laid') 1
('laid', 'over') 1
('the', 'ugly') 1
('ugly', 'face') 1
('face', 'of') 5
('of', 'violent') 1
('violent', 'coercion') 1
('coercion', '.') 1
('.', 'Are') 1
('Are', 'these') 1
('pictures', 'simply') 1
('of', 'power') 1
('power', ',') 2
('in', 'which') 17
('which', 'the') 12
('the', 'hard') 1
('hard', 'approach') 1
('approach', 'is') 1
('is', 'mixed') 1
('mixed', 'with') 1
('the', 'soft') 1
('soft', ',') 1
('Church', 'is') 1
('is', 'allied') 1
('allie

('threatening', 'castration') 1
('castration', ')') 1
('.', 'More') 7
('More', 'disturbingly') 1
('disturbingly', ',') 1
('in', 'The') 1
('The', 'Mask') 2
('Mask', '(') 1
('(', '1945') 1
('1945', ')') 1
('Fig.', '7') 3
('7', ')') 2
('she', 'throws') 1
('throws', 'doubt') 1
('doubt', 'on') 1
('on', 'too') 1
('too', 'straightforward') 1
('straightforward', 'a') 1
('a', 'reading') 1
('reading', 'of') 2
('her', 'self-portraits') 1
('self-portraits', 'as') 1
('as', 'revealing') 1
('revealing', 'of') 1
('her', 'inner') 1
('inner', 'emotions') 1
('emotions', '.') 1
('.', 'Where') 1
('Where', 'does') 1
('does', 'the') 2
('the', 'mask') 1
('mask', 'fall') 1
('fall', ':') 1
(':', 'Does') 1
('Does', 'not') 1
('the', 'ritual') 1
('ritual', 'repetition') 1
('repetition', 'of') 1
('of', 'those') 1
('those', 'familiar') 1
('familiar', 'features') 1
('features', '‘') 1
('‘', 'mask') 1
('mask', '’') 1
('’', 'far') 1
('more', 'than') 7
('than', 'it') 2
('it', 'uncovers') 1
('uncovers', '?') 1
('?', 'The

('coincides', 'with') 1
('the', 'decline') 1
('decline', 'and') 1
('and', 'ultimate') 2
('ultimate', 'collapse') 1
('collapse', 'of') 1
('the', 'Empire') 2
('Empire', ';') 1
(';', 'Spanish') 1
('Spanish', 'rule') 1
('rule', 'in') 2
('in', 'America') 4
('America', 'had') 1
('been', 'overthrown') 1
('overthrown', 'by') 1
('death', 'in') 1
('in', '1828') 1
('1828', '.') 1
('In', 'other') 3
(',', 'these') 4
('these', 'artists') 1
('artists', 'lived') 1
('lived', 'during') 1
('during', 'Latin') 1
('American', "'s") 1
("'s", 'formative') 1
('formative', 'years') 1
('years', '.') 2
('.', 'the') 1
('the', 'artists') 1
('who', 'interpreted') 1
('interpreted', 'their') 1
('their', 'historical') 1
('historical', 'reality') 1
('reality', 'on') 1
('on', 'to') 2
('to', 'canvas') 1
('the', 'importance') 3
('importance', 'in') 1
('America', 'of') 1
('art', 'itself') 2
('itself', 'should') 1
('should', 'not') 2
('be', 'underestimated') 1
('underestimated', ',') 1
('particularly', 'the') 2
('the', 'icon

('indisputably', 'greatest') 1
('greatest', 'figures') 1
('figures', 'in') 1
('the', 'whole') 3
('whole', 'of') 1
('of', 'Western') 3
('Western', 'art') 1
('art', 'devoted') 1
('devoted', 'the') 1
('the', 'better') 1
('better', 'part') 1
('his', 'life') 1
('life', 'to') 1
('to', 'sell-advancement') 1
('sell-advancement', ',') 1
('painting', 'the') 1
('King', ',') 1
('his', 'family') 1
('family', ',') 1
('their', 'attendant') 1
('attendant', 'dogs') 1
('dogs', ',') 1
(',', 'dwarves') 1
('dwarves', 'and') 1
('and', 'sycophants') 1
('sycophants', 'Certainly') 1
('Certainly', 'the') 1
('the', 'irony') 1
('irony', 'of') 2
('this', ',') 5
('this', 'inbred') 1
('inbred', 'family') 1
('family', 'of') 1
('often', 'considerable') 1
('considerable', 'mental') 1
('mental', 'as') 1
('as', 'physical') 1
('physical', 'fragility') 1
('fragility', 'should') 1
('have', 'controlled') 1
('controlled', 'the') 1
('the', 'destiny') 1
('destiny', 'of') 1
('of', 'so') 1
('so', 'vast') 1
('vast', 'an') 1
('an',

('marginalised', 'now') 1
('now', 'dominates') 1
('dominates', 'everything') 1
('everything', 'this') 1
('this', 'oxymoronic') 1
('oxymoronic', '‘') 1
('‘', 'radical') 1
('radical', 'orthodoxy') 1
('orthodoxy', '’') 1
('’', 'touches') 1
('touches', '.') 1
('And', 'it') 2
('it', 'touches') 1
('touches', 'a') 1
('a', 'lot') 1
('lot', '.') 1
('.', 'Making') 1
('Making', 'all') 1
('all', 'due') 1
('due', 'allowance') 1
('allowance', 'for') 1
('the', 'return') 1
('return', 'of') 1
('historical', 'tragedy') 1
('tragedy', 'as') 1
('as', 'farce') 1
('farce', ',') 1
(',', 'watching') 1
('watching', 'this') 1
('this', 'soporific') 1
('soporific', 'monolith') 1
('monolith', 'of') 1
('the', 'virtuous') 1
('virtuous', 'rise') 1
('rise', 'out') 1
('the', 'debris') 1
('debris', 'of') 1
('a', 'liberating') 1
('liberating', 'movement') 1
('movement', 'is') 2
('is', 'akin') 1
('akin', 'to') 1
('to', 'nothing') 1
('nothing', 'so') 1
('much', 'as') 1
('as', 'witnessing') 1
('witnessing', 'Bureaucracy') 1


('be', 'put') 1
('put', 'at') 1
('at', 'risk') 1
('risk', 'by') 1
('the', 'aesthetics') 1
('aesthetics', ',') 1
('relationship', 'as') 1
('as', 'each') 1
('each', 'of') 1
('its', 'components') 1
('components', 'will') 1
('be', 'inert') 1
('inert', '.') 1
('An', 'unpredictable') 1
('unpredictable', 'aesthetic') 1
('aesthetic', ',') 1
('said', ',') 1
('a', 'requirement') 1
('requirement', 'of') 1
('an', 'historical') 1
('materialism', 'adequate') 1
('to', 'its') 4
('political', 'task') 1
('task', '.') 1
('is', 'genuinely') 1
('genuinely', 'to') 1
('have', 'reciprocal') 1
('reciprocal', 'leverage') 1
('leverage', 'upon') 1
(',', 'certain') 2
('certain', 'problems') 1
('problems', 'have') 1
('be', 'faced') 1
('faced', '.') 1
('For', 'example') 6
('what', 'goes') 1
('goes', 'on') 1
('on', 'when') 1
('when', 'looking') 1
('looking', 'at') 1
('at', ',') 1
(',', 'say') 3
('Matisse', 'in') 1
('gallery', '.') 1
('is', 'easy') 1
('easy', 'to') 2
('time', 'spent') 2
('spent', 'considering') 2
('co

('impact', 'of') 1
('of', 'coloured') 1
('coloured', 'light') 1
('light', 'as') 1
('as', 'radically') 1
('radically', 'reorientating') 1
('reorientating', 'this') 1
('area', '.') 2
('With', 'perspective') 1
('key', 'figures') 1
('Revolution', 'are') 1
('the', 'heirs') 1
('heirs', 'to') 1
(',', 'we') 2
('we', 'encounter') 1
('encounter', 'Galileo') 1
('Galileo', 'as') 1
('a', 'bit-player') 1
('bit-player', 'who') 1
('who', 'corresponded') 1
('corresponded', 'with') 1
('with', 'Lodovico') 1
('Lodovico', 'Cigoli') 1
('Cigoli', 'over') 1
('over', 'his') 1
('his', 'lunar') 1
('lunar', 'and') 1
('and', 'sunspot') 1
('sunspot', 'observations') 1
('observations', '.') 1
('More', 'particularly') 1
('particularly', ',') 1
('Galileo', 'used') 1
('used', 'standard') 1
('standard', 'perspective') 1
('perspective', 'procedures') 1
('procedures', 'in') 1
('in', 'accounting') 1
('accounting', 'for') 1
('the', 'foreshortening') 1
('foreshortening', 'of') 1
('the', 'spots') 1
('spots', 'which') 1
('clai

('provides', 'is') 1
('the', 'missing') 2
('missing', 'chapter') 1
('chapter', 'in') 1
('the', 'secret') 1
('secret', 'history') 1
('the', 'cinema') 1
('cinema', ',') 1
('missing', 'link') 1
('link', 'between') 1
('between', 'Berlin') 1
('Berlin', 'Dada') 1
('postmodernism', '.') 2
('with', 'other') 1
('other', 'contributors') 1
('contributors', ',') 1
(',', 'exaggerated') 1
('exaggerated', 'claims') 1
('made', 'for') 1
('for', 'détournement') 1
('détournement', '(') 1
('the', 'communication') 1
('communication', 'which') 1
('which', 'contains') 1
('contains', 'its') 1
('own', 'critique') 1
('critique', ')') 1
(',', 'firstly') 1
('firstly', 'because') 1
('an', 'inheritance') 1
('inheritance', 'from') 1
('from', 'Cubism') 1
('Cubism', 'and') 1
('and', 'Dada') 1
('and', 'secondly') 1
('because', ',') 1
('exhibition', 'shows') 1
('its', 'deployment') 1
('deployment', 'by') 1
('‘', 'pro-situs') 1
('pro-situs', '’') 1
('has', 'made') 1
('it', 'a') 1
('a', 'commonplace') 1
('commonplace', ',